# DEMO: Diffusion prompts Generator

This notebook is a demonstrator of the generated prompts. It uses code from the `llm/prompts_generator.py`.

### Libraries

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
import json
import random

In [2]:
import sys
sys.path.append('..')
from llm.prompts_generator import load_model

### Model and tokenizer loader

In [3]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-AWQ"
model, tokenizer = load_model(model_name_or_path)


### Load class list as input of the prompts

In [4]:
# Load class list
with open('../llm/class_list.json', 'r') as file:
    class_list = json.load(file)

### Set seed for reproducibility
42 is moslty used as seed number and considered as the lucky number. 

In [5]:
seed = 42
random.seed(seed)

### Let's configuration the LLM model for the prompts generation

In [6]:
# Initialize the prompt template and tokenizer
prompt = ""
prompt_template=f'''{prompt}'''

tokens = tokenizer(
    prompt_template,
    return_tensors='pt'
).input_ids.cuda()

# Same as the "self-rewarding language models" paper
generation_params = {
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.95,
    "top_k": 40,
    "max_new_tokens": 512*8,
    "repetition_penalty": 1.1
}

# Using the text streamer to stream output one token at a time
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# SYSTEM_PROMPT to instruct the model on specificities
SYSTEM_PROMPT = "<s>[SYS] Generate very detailed scene description prompts for a text-to-image stable diffusion generative model. Focus on the specified class, give specific details of the scene with very comprehensive scene context description, describe the finest detail and avoid additional information, notes, suggestions and discussions. Only use the enumeration format when listing and no other format: 1. 2. 3. 4. 5. 6. 7. 8. 9. 10.[/SYS]"


### Random selection of 4 classes for the test

In [7]:
# Shuffle the classes to pick them randomly
class_keys = list(class_list.keys())
random.shuffle(class_keys)

selected_classes_keys = random.sample(class_keys, 4)

# Get the class names based on the selected keys
selected_classes = [class_list[class_key] for class_key in selected_classes_keys]

selected_classes

['hair drier', 'truck', 'toaster', 'cup']

5 prompts generation per selected class

In [8]:
for class_id in selected_classes_keys:
    class_name = class_list[class_id]

    # Combine SYSTEM_PROMPT and class-specific prompt
    prompt = f"{SYSTEM_PROMPT}<s>[INST] Give 10 distinct, long, very descriptive and very detailed prompts for a text-to-image diffusion model focusing mainly on: {class_name}. The {class_name} should be the main focus and element of the scene, very clear and easy to see and locate. The prompts should be exaustive in different cases, object numbers, scenarios, positions and orientations. Details of the background, the colors, the observed objects in the scene should also be given in a very exhaustive manner.[/INST]"

    tokens = tokenizer(
        prompt,
        return_tensors='pt'
    ).input_ids.cuda()

    print('\n' + class_name + '\n')
    generation_output = model.generate(
        tokens,
        pad_token_id=tokenizer.eos_token_id,
        streamer=streamer,
        **generation_params
    )


hair drier

1. A vibrant red kitchen, dominated by a glossy chrome hairdryer resting atop a marble countertop. The counter is cluttered with various cosmetics - a pink toothbrush stand, a bottle of shampoo, and a tub of body lotion. A blue striped towel is draped over the backless barstool, adding a pop of color against the white subway tile backsplash. A window behind the sink lets in soft sunlight, casting warm shadows over the room.

2. A bathroom scene, with a sleek silver hairdryer positioned on a porcelain shelf above an oversized bathtub. Steam rises from the tub, filling the air with a comforting aroma of lavender bath salts. A handheld mirror leans against the tiled wall, reflecting the steamy room and the hairdryer's metallic sheen. A small wooden stool is placed next to the tub, covered in a fluffy white bathrobe.

3. In a dimly lit dressing room, a hot pink hairdryer sits atop a plush velvet vanity chair. The room is adorned with ornate gold mirrors on the walls, and a lar